In [11]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *
import numpy as np
from itertools import combinations
from sklearn.model_selection import PredefinedSplit
import glob
pd.set_option('display.max_rows', None)

In [6]:
# Initialze the estimators
clf1 = RandomForestClassifier(random_state=42)
# clf2 = SVC(kernel='sigmoid',probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
# clf7 = GradientBoostingClassifier(random_state=42)
clf8 = MLPClassifier(max_iter=2000,random_state=42)
clf9 = SVC(kernel='poly',probability=True, random_state=42)
# clf10 = SVC(kernel='rbf',probability=True, random_state=42)


In [7]:
class ColumnExtractor(object):
    def __init__(self, cols=None):
        self.cols = cols
    def transform(self, X):
        return X[:,self.cols]
    def fit(self, X, y=None):
        return self
    def get_params(self,deep=True):
        return {"cols" : self.cols}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)

In [61]:
# Initiaze the hyperparameters for each dictionary
param1 = {}
# param1['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param1['feat_select__cols'] = [(0,2,4,6,8,10,12,14,16,18)]
param1['classifier__n_estimators'] = [10, 20, 30, 40]
param1['classifier__max_depth'] = [2, 3, 4]
param1['classifier__class_weight'] = [None, 'balanced']
# param1['classifier'] = [clf1]

param2 = {}
# param2['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
# param2['Classifier__kernel'] = [ 'linear']
param2['classifier__class_weight'] = [None, 'balanced']
param2['classifier__degree'] = [2,3]
# param2['classifier'] = [clf2]

param3 = {}
param3['feat_select__cols'] = [(0,2,4,6,8,10,12,14,16,18)]
param3['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, 'balanced']
# param3['classifier'] = [clf3]

param4 = {}
param4['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,4)], [])
param4['classifier__max_depth'] = [2,3,4,None]
param4['classifier__min_samples_split'] = [2,3,4]
param4['classifier__class_weight'] = [None, 'balanced']
# param4['classifier'] = [clf4]

param5 = {}
param5['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param5['classifier__n_neighbors'] = [2,3,4,5]
# param5['classifier'] = [clf5]

param6 = {}
param6['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
# param6['classifier'] = [clf6]

param7 = {}
param7['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param7['classifier__n_estimators'] = [10, 20, 30, 40]
param7['classifier__max_depth'] = [2, 3, 4]
# param7['classifier'] = [clf7]

param8 = {}
# param8['feat_select__cols'] = sum([list(combinations(np.arange(0,20,1), i)) for i in range(1,5)], [])
param8['feat_select__cols'] = [(0,2,4,6,8,10,12,14,16,18)]
param8['classifier__hidden_layer_sizes'] = [(10),(10,10),(10,10,10)]
param8['classifier__solver'] = ['adam', 'sgd']
param8['classifier__alpha'] = [0.0001, 0.0005,0.001,0.005,0.01,0.05]    
# param8['classifier'] = [clf8]

In [62]:
pipeline1 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf1)])
# pipeline2 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf2)])
pipeline3 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf3)])
pipeline4 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf4)])
pipeline5 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf5)])
pipeline6 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf6)])
# pipeline7 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf7)])
pipeline8 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf8)])
pipeline9 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf9)])
# pipeline10 = Pipeline([('feat_select',ColumnExtractor()),('classifier', clf10)])


In [26]:
glob.glob("X_train*")

['X_train_fft.csv',
 'X_train_hwt.csv',
 'X_train_laplacian.csv',
 'X_train_modified_laplacian.csv',
 'X_train_tenengrad.csv']

In [63]:
X_train_list = []
X_test_list = []

for files in glob.glob("X_train*"):
    X_train_list.append(pd.read_csv(files))
X_train = np.hstack(X_train_list)

for files in glob.glob("X_test*"):
    X_test_list.append(pd.read_csv(files))
X_test = np.hstack(X_test_list)

y_train = pd.read_csv("y_train_laplacian.csv").values.reshape(-1)
y_test = pd.read_csv("y_test_laplacian.csv").values.reshape(-1)
## Make binary labels
y_train1 = np.array(y_train)
y_train1[y_train1 > 0] = 1

y_test1 = np.array(y_test)
y_test1[y_test1 > 0] = 1

In [64]:
indices = np.zeros(len(y_train1) + len(y_test1))
indices[:len(y_train1)] = -1
Data_X = np.vstack([X_train,X_test])
Data_Y = np.hstack([y_train1,y_test1])

ps = PredefinedSplit(indices)
score = { "Accuracy":"balanced_accuracy" ,"Recall":"recall","Precision":"precision","F1_Score":"f1"}

In [65]:
grid1 = GridSearchCV(pipeline1, n_jobs=-1, param_grid=[param1],scoring=score,refit="F1_Score",cv=ps)
# grid2 = GridSearchCV(pipeline2, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)
grid3 = GridSearchCV(pipeline3, n_jobs=-1, param_grid=[param3],scoring=score,refit="F1_Score",cv=ps)
grid4 = GridSearchCV(pipeline4, n_jobs=-1, param_grid=[param4],scoring=score,refit="F1_Score",cv=ps)
grid5 = GridSearchCV(pipeline5, n_jobs=-1, param_grid=[param5],scoring=score,refit="F1_Score",cv=ps)
grid6 = GridSearchCV(pipeline6, n_jobs=-1, param_grid=[param6],scoring=score,refit="F1_Score",cv=ps)
# grid7 = GridSearchCV(pipeline7, n_jobs=-1, param_grid=[param7],scoring=score,refit="F1_Score",cv=ps)
grid8 = GridSearchCV(pipeline8, n_jobs=-1, param_grid=[param8],scoring=score,refit="F1_Score",cv=ps)
grid9 = GridSearchCV(pipeline9, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)
# grid10 = GridSearchCV(pipeline10, n_jobs=-1, param_grid=[param2],scoring=score,refit="F1_Score",cv=ps)

In [33]:
grids = [grid1,grid3,grid4,grid5,grid6,grid8,grid9]

In [ ]:
grid3.fit(Data_X,Data_Y)

In [ ]:
for pipe in grids:
    print("Yes Done")
    pipe.fit(Data_X,Data_Y)

In [ ]:
grid = grids[0]
print(grid.best_estimator_)
# print(pd.DataFrame(grid.cv_results_).iloc[grid.best_index_])

In [53]:
df = pd.DataFrame(grid1.cv_results_).drop(columns=["mean_fit_time","std_fit_time","mean_score_time","std_score_time","params",
                                             "mean_test_Accuracy","std_test_Accuracy","rank_test_Accuracy",
                                            "mean_test_Recall","std_test_Recall","rank_test_Recall",
                                            "mean_test_Precision","std_test_Precision","rank_test_Precision",
                                            "mean_test_F1_Score","std_test_F1_Score","rank_test_F1_Score"])

In [54]:
df["sum"] = df["split0_test_Accuracy"] + df["split0_test_Recall"] + df["split0_test_Precision"] + df["split0_test_F1_Score"]

In [56]:
df.sort_values("split0_test_F1_Score")[-5:][::-1]

,param_classifier__class_weight,param_classifier__max_depth,param_classifier__n_estimators,param_feat_select__cols,split0_test_Accuracy,split0_test_Recall,split0_test_Precision,split0_test_F1_Score,sum
11,None,4,40,"(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)",0.696429,0.964286,0.870968,0.915254,3.446936
10,None,4,30,"(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)",0.696429,0.964286,0.870968,0.915254,3.446936
2,None,2,30,"(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)",0.696429,0.964286,0.870968,0.915254,3.446936
3,None,2,40,"(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)",0.696429,0.964286,0.870968,0.915254,3.446936
7,None,3,40,"(0, 2, 4, 6, 8, 10, 12, 14, 16, 18)",0.696429,0.964286,0.870968,0.915254,3.446936


In [ ]:
fft - 0-3
hwt - 4-7
laplacian - 8-11
modified laplacian - 12-15
tenengrad - 16-19